BPE 代码实现设计

BPE算法的流程是，统计语料库中两两相邻的子词组成的子词对出现的频率，递归将最高频率的子词对组合成新的子词，并将新子词添加到词汇表中，直到词汇表大小达到预设的值。为了实现这一目标，我们有如下的需求：

1、 初始化子词对集，并记录每个子词对出现的频率
2、 取出最高频率的子词对，融合为新的子词后加入字典
3、 由于最高频率的子词对被融合，组成该子词对的两个子词参与促成的其它子词对数量将会减少，新子词参与组成的子词对则对应增加，需要据此修改子词对集的数据
4、在新旧子词对数量变化后，须再次找出新子词对集中频率最高的元素，重复上述的流程

接下来分析使用什么样的数据结构可以达成目标。

首先，由于需要统计每一个子词对的出现次数，容易想到使用哈希表实现。要求取出“频率最大”子词对，且不关心非最大频率子词对的频率顺序，可以想到用最大堆来快速获取最大值。

接下来，则是对子词对集进行数值修改、元素增删操作。第一个问题是，我们要如何找到受新子词影响的旧子词？最暴力的方法是从头到尾把语料库再遍历一次，但这样的效率显然不高。从语料库的特性我们可以知道，受影响的子词对必然在最大频率子词对的左右两侧，只要我们在统计频率时同时记录下该子词对出现过的所有位置，就可以借助索引快速找到对应位置，确定哪些子词对的频率会受影响，又有哪些新子词对会产生。也就是说，我们的哈希表里保存的是形如<byte-pair,[count,location_1, location_2,...,location_n]>的键值对

有了要修改和添加的对象，接下来就该考虑如何执行了。添加新节点到堆中不成问题，但我们还需要根据手头的子词对去找到最大堆中对应的节点，修改对应的频率值，同时维护堆的结构。根据键来查找元素是堆并不具备的功能，但用哈希表则可以轻松实现。我们可以在哈希表中存储形如<byte-pair,location_in_heap>的键值对，来将子词对在堆中的位置记录下来。这个位置可以是一个指针，也可以是数组的下标（如果我们用数组来实现堆）。在使用数组实现堆时，我们可以不使用任何额外的存储空间就完成对堆中任意一个节点的子节点的访问，从而在修改受影响的子词对的频率只会将对应节点下沉以维护堆的结构

